In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Reshape, Dropout, UpSampling2D
from tensorflow.keras.callbacks import LambdaCallback
import numpy as np
from tensorflow import keras
import tensorflow as tf
# import wandb
# from wandb.keras import WandbCallback
print(tf.__version__)

2.1.0


In [2]:
# hyperparameter
encoding_size = 32

In [3]:

# add noise func

def add_noise(x_train, x_test):
    # Function to add some random noise
    noise_factor = 1.0
    x_train_noisy = x_train + np.random.normal(loc=0.0, scale=noise_factor, size=x_train.shape) 
    x_test_noisy = x_test + np.random.normal(loc=0.0, scale=noise_factor, size=x_test.shape) 
    
    x_train_noisy = np.clip(x_train_noisy, 0., 1.)
    x_test_noisy = np.clip(x_test_noisy, 0., 1.)
    return x_train_noisy, x_test_noisy
# from tensorflow import kerast_noisy

In [4]:
# (x_train, _), (x_test, _) = fashion_mnist.load_data()
(x_train, _), (x_test, _) = mnist.load_data()
(x_train_noisy, x_test_noisy) = add_noise(x_train, x_test)
img_width = x_train.shape[1]
img_height = x_test.shape[2]
print("train shape:", x_train.shape)
print("test shape:", x_test.shape)
print("img_width:", img_width)
print("img_height:", img_height)

train shape: (60000, 28, 28)
test shape: (10000, 28, 28)
img_width: 28
img_height: 28


In [5]:
#normalize
x_train = x_train / 255.
x_test = x_test / 255.
# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(encoding_size, activation="relu"))
model.add(Dense(img_width*img_height, activation="sigmoid"))
model.add(Reshape((img_width, img_height)))
model.compile(loss='mse', optimizer='adam',
              metrics=['mse'])
model.summary()
keras.utils.plot_model(model, show_shapes=True, dpi=200, expand_nested=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25120     
_________________________________________________________________
dense_1 (Dense)              (None, 784)               25872     
_________________________________________________________________
reshape (Reshape)            (None, 28, 28)            0         
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [6]:
# add the image call back
import os
from datetime import datetime
logdir =  os.path.join("logs","image" + datetime.now().strftime("%Y%m%d-%H%M%S"))
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_img = tf.summary.create_file_writer(logdir + '/img')

def draw_input_output(epoch, logs):
    output = model.predict(x_test_noisy)
    print(output.shape)
    with file_writer_img.as_default():
        tf.summary.image("test_output", tf.reshape(output,[-1, 28, 28, 1]), step=epoch)
        tf.summary.image("test_noisy_input", tf.reshape(x_test_noisy,[-1, 28, 28, 1]), step=epoch)
        tf.summary.image("test_input", tf.reshape(x_test,[-1, 28, 28, 1]), step=epoch)
log_mg = keras.callbacks.LambdaCallback(on_epoch_end=draw_input_output)

In [7]:
# Fit the model

model.fit(x_train_noisy, x_train, epochs=30, validation_data=(x_test_noisy, x_test),
          callbacks=[tensorboard_callback, log_mg])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0393 - mse: 0.0393 - val_loss: 0.0256 - val_mse: 0.0256
Epoch 2/30
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0231 - mse: 0.0231 - val_loss: 0.0209 - val_mse: 0.0209
Epoch 3/30
60000/60000 [==============================] - 4s 69us/sample - loss: 0.0208 - mse: 0.0208 - val_loss: 0.0201 - val_mse: 0.0201
Epoch 4/30
60000/60000 [==============================] - 4s 67us/sample - loss: 0.0204 - mse: 0.0204 - val_loss: 0.0201 - val_mse: 0.0201
Epoch 5/30
60000/60000 [==============================] - 4s 65us/sample - loss: 0.0202 - mse: 0.0202 - val_loss: 0.0198 - val_mse: 0.0198
Epoch 6/30
60000/60000 [==============================] - 4s 69us/sample - loss: 0.0200 - mse: 0.0200 - val_loss: 0.0196 - val_mse: 0.0196
Epoch 7/30
60000/60000 [==============================] - 4s 68us/sample - loss: 0.0199 - mse: 0.0199 - val_loss: 0.